# Automatically calculating number of rays to shoot based on user's desired uncertainty

### Fractional uncertainty: $\delta$

### Number of rays in source plane per pixel of width $dx$:
$n_{source} = \dfrac{\delta^{-2}}{dx^2}$

### This gets transformed into the image plane, where the area of the pixel is scaled up by $A$, the magnification.
### The new number of rays per area in the image plane is:
$n_{image} = \dfrac{\delta^{-2}}{A dx^2}$


### Magnification as a function of seperation from single lens:
$A(u) = \dfrac{u^2 + 2}{u \sqrt{u^2 + 4}}$

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

from IRSMicroLensing import IRSCaustics as IRSC

In [2]:
def A(u):
    return (u**2 + 2) / (u * np.sqrt(u**2 + 4))

In [3]:
lens_att = [
    [0, 0, 0.01, 1],
    [0.8, 0, 0.01, 0.001]
]

num_theta = 5000
num_r = 2 * num_theta

param_dict_1 = {'pixels': 1000, 'ang_width': 'auto', 'lens_att': lens_att, 'thickness': 'auto', 'num_r': num_r, 'num_theta': num_theta}

mag_map_1 = IRSC.IRSCaustics(annulus_param_dict=param_dict_1)
print(mag_map_1.param_dict)
print(mag_map_1.y_plus, mag_map_1.y_minus)

mag_map_1.plot()

plt.show()

{'pixels': 1000, 'ang_width': np.float64(0.07680319680319682), 'lens_att': [[0, 0, 0.01, 1], [0.8, 0, 0.01, 0.001]], 'thickness': np.float64(0.059738867403980045), 'num_r': 10000, 'num_theta': 5000, 'ang_res': np.float64(7.680319680319682e-05), 'pixel_density': np.float64(13020.29136316337), 'dr': np.float64(5.973886740398004e-06), 'dtheta': 0.072, 'num_rays': 50000000}
1.030315425782361 -0.970576558378381
Creating mesh grid: 1.178 seconds
1 1.1756775379180908
2 1.0547726154327393
3 5.2928924560546875e-05
4 6.9141387939453125e-06
5 3.5762786865234375e-06
6 0.00025844573974609375
7 3.0054242610931396
9 2.47955322265625e-05
10 4.0531158447265625e-06
Calculating source pixels: 5.313 seconds
Calculating indices of translated pixel after deflection: 2.927 seconds


: 